# Load as torch model

Loads the base neural network model and the `sim_to_nn` transformers from their respective files. They are combined to create a `TransformedModel` and tested on a small set of simulation data.

In [1]:
import os

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")       

True

In [6]:
import mlflow
import mlflow.pytorch
import torch
import matplotlib.pyplot as plt

In [7]:
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
mlflow.set_experiment("lcls-injector-ML")

<Experiment: artifact_location='s3://mlflow-bucket/5', creation_time=1746036982208, experiment_id='5', last_update_time=1746036982208, lifecycle_stage='active', name='lcls-injector-ML', tags={}>

In [8]:
# load base model and transformers
model = torch.load("../model/model.pt")
input_sim_to_nn = torch.load("../model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load("../model/output_sim_to_nn.pt")

## Create model

In [9]:
# define transformed model class
class TransformedModel(torch.nn.Module):
    def __init__(self, model, input_transformer, output_transformer):
        super(TransformedModel, self).__init__()
        self.model = model
        self.input_transformer = input_transformer
        self.output_transformer = output_transformer
    
    def forward(self, x):
        x = self.input_transformer(x)
        x = self.model(x)
        x = self.output_transformer.untransform(x)
        return x

In [10]:
# create transformed model
transformed_model = TransformedModel(
    model=model, 
    input_transformer=input_sim_to_nn,
    output_transformer=output_sim_to_nn,
).to(torch.double)
transformed_model

TransformedModel(
  (model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ELU(alpha=1.0)
    (4): Dropout(p=0.05, inplace=False)
    (5): Linear(in_features=200, out_features=200, bias=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=200, out_features=300, bias=True)
    (9): ELU(alpha=1.0)
    (10): Dropout(p=0.05, inplace=False)
    (11): Linear(in_features=300, out_features=300, bias=True)
    (12): ELU(alpha=1.0)
    (13): Dropout(p=0.05, inplace=False)
    (14): Linear(in_features=300, out_features=200, bias=True)
    (15): ELU(alpha=1.0)
    (16): Dropout(p=0.05, inplace=False)
    (17): Linear(in_features=200, out_features=100, bias=True)
    (18): ELU(alpha=1.0)
    (19): Dropout(p=0.05, inplace=False)
    (20): Linear(in_features=100, out_features=100, bias=True)
    (21): ELU(alpha=1.0)
    (22): Linear(in_

## Test on example data

In [13]:
# load example data and calculate predictions
import torch.nn.functional as F

with mlflow.start_run(run_name="lcls-injector-ML Run6"):
    mlflow.log_param("model_type", str(type(model)))
    mlflow.log_param("input_transformer_type", str(type(input_sim_to_nn)))
    mlflow.log_param("output_transformer_type", str(type(output_sim_to_nn)))
    
    # Load input/output data
    inputs_small = torch.load("../info/inputs_small.pt")
    outputs_small = torch.load("../info/outputs_small.pt")

    with torch.no_grad():
        predictions = transformed_model(inputs_small)

    mae = torch.mean(torch.abs(predictions - outputs_small)).item()
    mlflow.log_metric("mean_absolute_error", mae)
    mlflow.pytorch.log_model(transformed_model, artifact_path="transformed_model")

    # Plot predictions vs ground truth
    nrows, ncols = 3, 2
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 15))
    for i in range(nrows * ncols):
        ax_i = ax[i // ncols, i % ncols]
        if i < outputs_small.shape[1]:
            sort_idx = torch.argsort(outputs_small[:, i])
            x_axis = torch.arange(outputs_small.shape[0])
            ax_i.plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
            ax_i.plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
            ax_i.legend()
    ax[-1, -1].axis('off')
    fig.tight_layout()

    # Save and log plot
    plot_path = "comparison_plot.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()


2025/05/02 10:03:42 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/05/02 10:03:42 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/05/02 10:05:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lcls-injector-ML Run6 at: https://ard-mlflow.slac.stanford.edu/#/experiments/5/runs/8efaa5dc63b04bf2831d2ac7a985b9e2
🧪 View experiment at: https://ard-mlflow.slac.stanford.edu/#/experiments/5


2025/05/02 10:05:15 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/05/02 10:05:16 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [16]:
import mlflow

if mlflow.active_run() is not None:
    mlflow.end_run()


🏃 View run mercurial-horse-202 at: https://ard-mlflow.slac.stanford.edu/#/experiments/5/runs/75f39c7e5c8441299256edb716d21717
🧪 View experiment at: https://ard-mlflow.slac.stanford.edu/#/experiments/5


2025/04/30 11:31:16 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/04/30 11:31:16 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
